In [1]:
import gzip, pickle
from aif360.algorithms.preprocessing import DisparateImpactRemover
from ExistingCombosFnR import load_data, get_Xsy
from aif360.datasets import AdultDataset
from sklearn.preprocessing import MinMaxScaler

/Users/haipei/.pyenv/versions/anaconda3-2020.11/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/haipei/.pyenv/versions/anaconda3-2020.11/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/haipei/.pyenv/versions/anaconda3-2020.11/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/

In [2]:
# train, test, privileged_groups, unprivileged_groups = load_data('adult','sex')
protected = 'sex'
ad = AdultDataset(protected_attribute_names=[protected],
    privileged_classes=[['Male']], categorical_features=[],
    features_to_keep=['age', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week'])
scaler = MinMaxScaler(copy=False)
test, train = ad.split([16281])
train.features = scaler.fit_transform(train.features)
test.features = scaler.fit_transform(test.features)

In [3]:
from aif360.algorithms.preprocessing import DisparateImpactRemover
di = DisparateImpactRemover(repair_level=1.0)

In [4]:
train_tred = di.fit_transform(train)
test_tred = di.fit_transform(test)

In [5]:
X_, s_, y_, weight_ = get_Xsy(train_tred, attr='sex')
X, s, y, weight_ = get_Xsy(train, attr='sex')

In [8]:
X_[:10]

array([[0.30136986, 0.8       , 0.02174022, 0.        , 0.40816327],
       [0.45205479, 0.8       , 0.        , 0.        , 0.12244898],
       [0.28767123, 0.53333333, 0.        , 0.        , 0.40816327],
       [0.49315068, 0.4       , 0.        , 0.        , 0.40816327],
       [0.15068493, 0.8       , 0.        , 0.        , 0.47959184],
       [0.2739726 , 0.86666667, 0.        , 0.        , 0.47959184],
       [0.43835616, 0.26666667, 0.        , 0.        , 0.18367347],
       [0.47945205, 0.53333333, 0.        , 0.        , 0.44897959],
       [0.19178082, 0.86666667, 0.14084141, 0.        , 0.60204082],
       [0.34246575, 0.8       , 0.05178052, 0.        , 0.40816327]])

In [9]:
X[:10]

array([[0.30136986, 0.8       , 0.02174022, 0.        , 0.39795918],
       [0.45205479, 0.8       , 0.        , 0.        , 0.12244898],
       [0.28767123, 0.53333333, 0.        , 0.        , 0.39795918],
       [0.49315068, 0.4       , 0.        , 0.        , 0.39795918],
       [0.15068493, 0.8       , 0.        , 0.        , 0.39795918],
       [0.2739726 , 0.86666667, 0.        , 0.        , 0.39795918],
       [0.43835616, 0.26666667, 0.        , 0.        , 0.15306122],
       [0.47945205, 0.53333333, 0.        , 0.        , 0.44897959],
       [0.19178082, 0.86666667, 0.14084141, 0.        , 0.5       ],
       [0.34246575, 0.8       , 0.05178052, 0.        , 0.39795918]])

In [53]:
from sklearn.linear_model import LogisticRegression
from metric import Metric
model = LogisticRegression(max_iter=1000)

In [54]:
model.fit(X_,y_)
y_pred_ = model.predict(X_)
metric = Metric(true=y_,pred=y_pred_)
print(metric.accuracy(), metric.positive_disparity(s=s_, absolute=False))

0.8093424649120113 -0.049395762907462046


In [55]:
model.fit(X,y)
y_pred = model.predict(X)
metric = Metric(true=y,pred=y_pred)
print(metric.accuracy(), metric.positive_disparity(s=s, absolute=False))

0.8119836614354596 -0.08158519240448524
